In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rishi\\Desktop\\data\\kaggle\\NLPTextSummarization\\NLPTextSummarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rishi\\Desktop\\data\\kaggle\\NLPTextSummarization\\NLPTextSummarization'

Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig: #from config.yaml file the value is the variable while the key is the output format type
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

#from the above code whenever root_dir or any of the mentioned variables will be called it will return the assigned path as per the config.yaml file

In [6]:
from textSummarizer.constants import * #since we want to access all the constants
from textSummarizer.utils.common import read_yaml, create_directories #these files have already created inside common src/utils/common

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) #this is a dictionary pair call only possible because of ConfigBox

    #this code block reads the constants using the function we have desigend within common.py present inside utils 
    #(eg: procedure to read the yaml file)
    # it also saves the file inside the artifacts folder

    

    def get_data_ingestion_config(self) -> DataIngestionConfig: # the return type is set to DataIngesionConfig which has been stated above in code block 1
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

Update components

In [8]:
import os
import urllib.request as request #this is used to download the data from the URL
import zipfile # to perform the unzip operation after data download
from textSummarizer.logging import logger # cusotm logger
from textSummarizer.utils.common import get_size # this module is created inside common.py

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

Update the pipeline

In [11]:
# the code block reads the file based on the mentioned variables 
#and the method as defined above and srotes them in variables and performs the download and extraction of zip file
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-26 23:40:31,203: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-26 23:40:31,218: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-26 23:40:31,220: INFO: common: created directory at: artifacts]
[2023-12-26 23:40:31,221: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-26 23:40:36,484: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1818:3C94:247B1B:2FAE24:658B3A40
Accept-Ranges: bytes
Date: Tue, 26 Dec 2023 20:40:35 GMT
Via: 1.1 varnish
X-Served-By: cache-fjr990024-FJR
X-Cache: MI